In [1]:
critic_type = 'inpainting'

In [17]:
model_list = ['5.20_inpaint_transfer_sampled_3.pkl', '5.20_inpaint_imagenet_sampled_3.pkl', 
              '5.21_inpaint_transfer_toddler_sampled_3.pkl', '5.21_inpaint_imagenet_toddler_sampled_3.pkl']
tag_list = ['transfer_sampled', 'imagenet_sampled', 'transfer_toddler_sampled', 'imagenet_toddler_sampled']

In [3]:
import sys
sys.path.append("../../../")

In [4]:
import fastai
from torchvision import transforms
from fastai import *
from fastai.vision import *
from fastai.callbacks import *
from fastai.script import *

from model.metrics import *

from skimage.measure import compare_psnr, compare_ssim

In [5]:
from data.load import get_data

# Choose GPU Device

In [6]:
torch.cuda.set_device(1)

# Load Data

### Parameters for Data Loading

In [7]:
data_pth = Path('/home/alaa/Dropbox/BPHO Staff/USF')  # path to dataset
lr_dir = f'EM/training/trainsets/lr/'  # input images
hr_dir = f'EM/training/trainsets/hr/'  # target images
model_path = data_pth/f'EM/models/self_sv/{critic_type}'
test_pth = data_pth/f'EM/testing/LR/real-world_SEM'
infer_pth = data_pth/f'EM/testing/self_sv/{critic_type}'

In [8]:
from PIL import Image
img = Image.open(list(test_pth.glob('*.tif'))[0])
img.size

(125, 125)

In [9]:
bs = 8  # batch size
size = 500  # image size

In [10]:
# Store dataset in a databunch object 
data = get_data(data_pth, lr_dir, hr_dir, bs, 
                in_sz=size, out_sz=size, max_zoom=1.)

In [11]:
from model.metrics import psnr, ssim
superres_metrics = [F.mse_loss, psnr, ssim]

# Modelling

## Load Model

In [12]:
arch = models.resnet34
wd = 1e-3
learn = unet_learner(data, arch, wd=wd, 
                     #loss_func=feat_loss,
                     loss_func=F.l1_loss,
                     metrics=superres_metrics,
                     #callback_fns=LossMetrics, 
                     blur=True, norm_type=NormType.Weight, model_dir=model_path)
gc.collect()

0

## Inference

In [13]:
test_set = ImageList.from_folder(test_pth, convert_mode='L')
learn.data.add_test(test_set, tfm_y=False)

In [14]:
mean, std = imagenet_stats # norm and denorm

In [15]:
def model_inference(learner=learn, folder_name=Path("CHANGEME/real-world_SEM"), img_tag=f"CHANGEME"):
    dir_name = infer_pth/folder_name
    if not os.path.isdir(dir_name): dir_name.mkdir(parents=True, exist_ok=True)
    print(dir_name)
    
    for img, img_name in zip(learner.data.test_ds, learner.data.test_ds.items):
        for i in range(3):
            img[0].data[i] -= mean[i]
            img[0].data[i] /= std[i]
        pred = learner.predict(img[0])
        for i in range(3):
            pred[1][i] *= std[i]
            pred[1][i] += mean[i]
        pred_img = transforms.ToPILImage()(pred[1][0])
        pred_name = dir_name/img_name.name.replace('lr', img_tag)
        print(pred_name)
        pred_img.save(pred_name)
        print(f"Performed inference on {img_name.stem}, file saved as {pred_name}")
    print("Model Inference Complete")

In [18]:
for model_name, img_tag in zip(model_list, tag_list):
    learn.model.load_state_dict(torch.load(model_path/model_name))
    model_inference(folder_name=Path(f"{model_name}/real-world_SEM"), img_tag=img_tag)

/home/alaa/Dropbox/BPHO Staff/USF/EM/testing/self_sv/inpainting/5.20_inpaint_transfer_sampled_3.pkl/real-world_SEM
/home/alaa/Dropbox/BPHO Staff/USF/EM/testing/self_sv/inpainting/5.20_inpaint_transfer_sampled_3.pkl/real-world_SEM/realword_SEM_test_transfer_sampled_05.tif
Performed inference on realword_SEM_test_lr_05, file saved as /home/alaa/Dropbox/BPHO Staff/USF/EM/testing/self_sv/inpainting/5.20_inpaint_transfer_sampled_3.pkl/real-world_SEM/realword_SEM_test_transfer_sampled_05.tif
/home/alaa/Dropbox/BPHO Staff/USF/EM/testing/self_sv/inpainting/5.20_inpaint_transfer_sampled_3.pkl/real-world_SEM/realword_SEM_test_transfer_sampled_38.tif
Performed inference on realword_SEM_test_lr_38, file saved as /home/alaa/Dropbox/BPHO Staff/USF/EM/testing/self_sv/inpainting/5.20_inpaint_transfer_sampled_3.pkl/real-world_SEM/realword_SEM_test_transfer_sampled_38.tif
/home/alaa/Dropbox/BPHO Staff/USF/EM/testing/self_sv/inpainting/5.20_inpaint_transfer_sampled_3.pkl/real-world_SEM/realword_SEM_test_

/home/alaa/Dropbox/BPHO Staff/USF/EM/testing/self_sv/inpainting/5.20_inpaint_transfer_sampled_3.pkl/real-world_SEM/realword_SEM_test_transfer_sampled_21.tif
Performed inference on realword_SEM_test_lr_21, file saved as /home/alaa/Dropbox/BPHO Staff/USF/EM/testing/self_sv/inpainting/5.20_inpaint_transfer_sampled_3.pkl/real-world_SEM/realword_SEM_test_transfer_sampled_21.tif
/home/alaa/Dropbox/BPHO Staff/USF/EM/testing/self_sv/inpainting/5.20_inpaint_transfer_sampled_3.pkl/real-world_SEM/realword_SEM_test_transfer_sampled_26.tif
Performed inference on realword_SEM_test_lr_26, file saved as /home/alaa/Dropbox/BPHO Staff/USF/EM/testing/self_sv/inpainting/5.20_inpaint_transfer_sampled_3.pkl/real-world_SEM/realword_SEM_test_transfer_sampled_26.tif
/home/alaa/Dropbox/BPHO Staff/USF/EM/testing/self_sv/inpainting/5.20_inpaint_transfer_sampled_3.pkl/real-world_SEM/realword_SEM_test_transfer_sampled_06.tif
Performed inference on realword_SEM_test_lr_06, file saved as /home/alaa/Dropbox/BPHO Staff

/home/alaa/Dropbox/BPHO Staff/USF/EM/testing/self_sv/inpainting/5.20_inpaint_imagenet_sampled_3.pkl/real-world_SEM/realword_SEM_test_imagenet_sampled_41.tif
Performed inference on realword_SEM_test_lr_41, file saved as /home/alaa/Dropbox/BPHO Staff/USF/EM/testing/self_sv/inpainting/5.20_inpaint_imagenet_sampled_3.pkl/real-world_SEM/realword_SEM_test_imagenet_sampled_41.tif
/home/alaa/Dropbox/BPHO Staff/USF/EM/testing/self_sv/inpainting/5.20_inpaint_imagenet_sampled_3.pkl/real-world_SEM/realword_SEM_test_imagenet_sampled_10.tif
Performed inference on realword_SEM_test_lr_10, file saved as /home/alaa/Dropbox/BPHO Staff/USF/EM/testing/self_sv/inpainting/5.20_inpaint_imagenet_sampled_3.pkl/real-world_SEM/realword_SEM_test_imagenet_sampled_10.tif
/home/alaa/Dropbox/BPHO Staff/USF/EM/testing/self_sv/inpainting/5.20_inpaint_imagenet_sampled_3.pkl/real-world_SEM/realword_SEM_test_imagenet_sampled_28.tif
Performed inference on realword_SEM_test_lr_28, file saved as /home/alaa/Dropbox/BPHO Staff

/home/alaa/Dropbox/BPHO Staff/USF/EM/testing/self_sv/inpainting/5.20_inpaint_imagenet_sampled_3.pkl/real-world_SEM/realword_SEM_test_imagenet_sampled_11.tif
Performed inference on realword_SEM_test_lr_11, file saved as /home/alaa/Dropbox/BPHO Staff/USF/EM/testing/self_sv/inpainting/5.20_inpaint_imagenet_sampled_3.pkl/real-world_SEM/realword_SEM_test_imagenet_sampled_11.tif
/home/alaa/Dropbox/BPHO Staff/USF/EM/testing/self_sv/inpainting/5.20_inpaint_imagenet_sampled_3.pkl/real-world_SEM/realword_SEM_test_imagenet_sampled_31.tif
Performed inference on realword_SEM_test_lr_31, file saved as /home/alaa/Dropbox/BPHO Staff/USF/EM/testing/self_sv/inpainting/5.20_inpaint_imagenet_sampled_3.pkl/real-world_SEM/realword_SEM_test_imagenet_sampled_31.tif
/home/alaa/Dropbox/BPHO Staff/USF/EM/testing/self_sv/inpainting/5.20_inpaint_imagenet_sampled_3.pkl/real-world_SEM/realword_SEM_test_imagenet_sampled_17.tif
Performed inference on realword_SEM_test_lr_17, file saved as /home/alaa/Dropbox/BPHO Staff

/home/alaa/Dropbox/BPHO Staff/USF/EM/testing/self_sv/inpainting/5.21_inpaint_transfer_toddler_sampled_3.pkl/real-world_SEM/realword_SEM_test_transfer_toddler_sampled_09.tif
Performed inference on realword_SEM_test_lr_09, file saved as /home/alaa/Dropbox/BPHO Staff/USF/EM/testing/self_sv/inpainting/5.21_inpaint_transfer_toddler_sampled_3.pkl/real-world_SEM/realword_SEM_test_transfer_toddler_sampled_09.tif
/home/alaa/Dropbox/BPHO Staff/USF/EM/testing/self_sv/inpainting/5.21_inpaint_transfer_toddler_sampled_3.pkl/real-world_SEM/realword_SEM_test_transfer_toddler_sampled_02.tif
Performed inference on realword_SEM_test_lr_02, file saved as /home/alaa/Dropbox/BPHO Staff/USF/EM/testing/self_sv/inpainting/5.21_inpaint_transfer_toddler_sampled_3.pkl/real-world_SEM/realword_SEM_test_transfer_toddler_sampled_02.tif
/home/alaa/Dropbox/BPHO Staff/USF/EM/testing/self_sv/inpainting/5.21_inpaint_transfer_toddler_sampled_3.pkl/real-world_SEM/realword_SEM_test_transfer_toddler_sampled_23.tif
Performed i

/home/alaa/Dropbox/BPHO Staff/USF/EM/testing/self_sv/inpainting/5.21_inpaint_transfer_toddler_sampled_3.pkl/real-world_SEM/realword_SEM_test_transfer_toddler_sampled_01.tif
Performed inference on realword_SEM_test_lr_01, file saved as /home/alaa/Dropbox/BPHO Staff/USF/EM/testing/self_sv/inpainting/5.21_inpaint_transfer_toddler_sampled_3.pkl/real-world_SEM/realword_SEM_test_transfer_toddler_sampled_01.tif
/home/alaa/Dropbox/BPHO Staff/USF/EM/testing/self_sv/inpainting/5.21_inpaint_transfer_toddler_sampled_3.pkl/real-world_SEM/realword_SEM_test_transfer_toddler_sampled_13.tif
Performed inference on realword_SEM_test_lr_13, file saved as /home/alaa/Dropbox/BPHO Staff/USF/EM/testing/self_sv/inpainting/5.21_inpaint_transfer_toddler_sampled_3.pkl/real-world_SEM/realword_SEM_test_transfer_toddler_sampled_13.tif
/home/alaa/Dropbox/BPHO Staff/USF/EM/testing/self_sv/inpainting/5.21_inpaint_transfer_toddler_sampled_3.pkl/real-world_SEM/realword_SEM_test_transfer_toddler_sampled_34.tif
Performed i

/home/alaa/Dropbox/BPHO Staff/USF/EM/testing/self_sv/inpainting/5.21_inpaint_imagenet_toddler_sampled_3.pkl/real-world_SEM/realword_SEM_test_imagenet_toddler_sampled_20.tif
Performed inference on realword_SEM_test_lr_20, file saved as /home/alaa/Dropbox/BPHO Staff/USF/EM/testing/self_sv/inpainting/5.21_inpaint_imagenet_toddler_sampled_3.pkl/real-world_SEM/realword_SEM_test_imagenet_toddler_sampled_20.tif
/home/alaa/Dropbox/BPHO Staff/USF/EM/testing/self_sv/inpainting/5.21_inpaint_imagenet_toddler_sampled_3.pkl/real-world_SEM/realword_SEM_test_imagenet_toddler_sampled_14.tif
Performed inference on realword_SEM_test_lr_14, file saved as /home/alaa/Dropbox/BPHO Staff/USF/EM/testing/self_sv/inpainting/5.21_inpaint_imagenet_toddler_sampled_3.pkl/real-world_SEM/realword_SEM_test_imagenet_toddler_sampled_14.tif
/home/alaa/Dropbox/BPHO Staff/USF/EM/testing/self_sv/inpainting/5.21_inpaint_imagenet_toddler_sampled_3.pkl/real-world_SEM/realword_SEM_test_imagenet_toddler_sampled_32.tif
Performed i

/home/alaa/Dropbox/BPHO Staff/USF/EM/testing/self_sv/inpainting/5.21_inpaint_imagenet_toddler_sampled_3.pkl/real-world_SEM/realword_SEM_test_imagenet_toddler_sampled_25.tif
Performed inference on realword_SEM_test_lr_25, file saved as /home/alaa/Dropbox/BPHO Staff/USF/EM/testing/self_sv/inpainting/5.21_inpaint_imagenet_toddler_sampled_3.pkl/real-world_SEM/realword_SEM_test_imagenet_toddler_sampled_25.tif
/home/alaa/Dropbox/BPHO Staff/USF/EM/testing/self_sv/inpainting/5.21_inpaint_imagenet_toddler_sampled_3.pkl/real-world_SEM/realword_SEM_test_imagenet_toddler_sampled_03.tif
Performed inference on realword_SEM_test_lr_03, file saved as /home/alaa/Dropbox/BPHO Staff/USF/EM/testing/self_sv/inpainting/5.21_inpaint_imagenet_toddler_sampled_3.pkl/real-world_SEM/realword_SEM_test_imagenet_toddler_sampled_03.tif
/home/alaa/Dropbox/BPHO Staff/USF/EM/testing/self_sv/inpainting/5.21_inpaint_imagenet_toddler_sampled_3.pkl/real-world_SEM/realword_SEM_test_imagenet_toddler_sampled_16.tif
Performed i

# Model Evaluation

In [19]:
target_dirname = data_pth/f'EM/testing/HR/real-world_SEM/'       # target images
target_files = sorted(list(target_dirname.glob('*.tif')))
print('Processing '+str(len(target_files))+' files...')

Processing 42 files...


In [20]:
def evaluate_model(pred_files, hr_files):
    psnr_scores = {}
    ssim_scores = {}
    for pred_fname, targ_fname in zip(pred_files, hr_files):
        pred_img = PIL.Image.open(pred_fname)
        targ_img = PIL.Image.open(targ_fname)
        
        pred = np.asarray(pred_img, np.uint8)
        targ = np.asarray(targ_img, np.uint8)
        
        psnr_score = compare_psnr(targ, pred, data_range=255.)
        psnr_scores[targ_fname] = psnr_score
        
        ssim_score = compare_ssim(targ, pred, data_range=255.)
        ssim_scores[targ_fname] = ssim_score
    return psnr_scores, ssim_scores

In [21]:
results = {}
mean_psnr = "mean psnr"
mean_ssim = "mean ssim"
for model_name, img_tag in zip(model_list, tag_list):
    inference_dirname = infer_pth/f'{model_name}/real-world_SEM/'  
    inference_files = sorted(list(inference_dirname.glob('*.tif')))
    results[img_tag] = {}
    results[img_tag]['psnr'], results[img_tag]['ssim'] = evaluate_model(inference_files, target_files)
    results[img_tag]['mean psnr'] = np.mean([results[img_tag]['psnr'][f] for f in results[img_tag]['psnr'].keys()])
    results[img_tag]['mean ssim'] = np.mean([results[img_tag]['ssim'][f] for f in results[img_tag]['ssim'].keys()])
    print(f'{img_tag} \n \t Mean PSNR: {results[img_tag][mean_psnr]:.3f} \n \t Mean SSIM: {results[img_tag][mean_ssim]:.3f}')

transfer_sampled 
 	 Mean PSNR: 24.186 
 	 Mean SSIM: 0.500
imagenet_sampled 
 	 Mean PSNR: 23.763 
 	 Mean SSIM: 0.518
transfer_toddler_sampled 
 	 Mean PSNR: 23.727 
 	 Mean SSIM: 0.500
imagenet_toddler_sampled 
 	 Mean PSNR: 24.525 
 	 Mean SSIM: 0.496


# Visualization

## Toddler: the critic is trained on 10% data

## Sampled: the generator is trained on 10% data

In [22]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
np.set_printoptions(linewidth=110)

# Set size for visualizations
fig_size = plt.rcParams["figure.figsize"]  # Get current size
print("Current size:", fig_size)
 
# Set figure width to 12 and height to 9
fig_size[0] = 30
fig_size[1] = 24
plt.rcParams["figure.figsize"] = fig_size

Current size: [6.0, 4.0]


In [23]:
model_1_name = model_list[0]
model_2_name = model_list[1]

dir1 = f'{model_1_name}/real-world_SEM/'  # pssr images
dir2 = f'{model_2_name}/real-world_SEM/'  # our images

# Modify accordingly
pth1 = infer_pth/dir1
pth2 = infer_pth/dir2

model1_files = sorted(list(pth1.glob('*.tif')))
model2_files = sorted(list(pth2.glob('*.tif')))

print('Processing '+str(len(model1_files))+' files...')

Processing 42 files...


In [24]:
def visualize_sample(img_fname1, img_fname2, targ_fname, title1=model_1_name, title2="Target", title3=model_2_name):
    model1_im = PIL.Image.open(img_fname1) # load image to tensor
    targ_im = PIL.Image.open(targ_fname)
    model2_im = PIL.Image.open(img_fname2)
    
    model1_arr = np.asarray(model1_im, np.uint8)
    model2_arr = np.asarray(model2_im, np.uint8)
    targ_arr = np.asarray(targ_im, np.uint8)
    
    model1_psnr = compare_psnr(targ_arr, model1_arr, data_range=255.)
    model1_ssim = compare_ssim(targ_arr, model1_arr, data_range=255.)
    model2_psnr = compare_psnr(targ_arr, model2_arr, data_range=255.)
    model2_ssim = compare_ssim(targ_arr, model2_arr, data_range=255.)
    f, axarr = plt.subplots(1,3) # create visualizations
    axarr[0].imshow(np.squeeze(model1_arr), cmap=plt.cm.gray) # visualize image tensor
    axarr[0].set_title(title1)
    axarr[0].set_xlabel(f"PSNR: {model1_psnr:.2f}, SSIM: {model1_ssim:.2f}")
    axarr[1].imshow(np.squeeze(targ_arr), cmap=plt.cm.gray) # visualize original image file
    axarr[1].set_title(title2)
    axarr[2].imshow(np.squeeze(model2_arr), cmap=plt.cm.gray) # visualize image tensor
    axarr[2].set_title(title3)
    axarr[2].set_xlabel(f"PSNR: {model2_psnr:.2f}, SSIM: {model2_ssim:.2f}")
    plt.show() # show visualization

In [25]:
@interact
def show_sample(sample=33):
    return visualize_sample(model1_files[sample], model2_files[sample], target_files[sample])

interactive(children=(IntSlider(value=33, description='sample', max=99, min=-33), Output()), _dom_classes=('wi…

In [26]:
model_3_name = model_list[2]
model_4_name = model_list[3]

dir3 = f'{model_3_name}/real-world_SEM/'  # pssr images
dir4 = f'{model_4_name}/real-world_SEM/'  # our images

# Modify accordingly
pth3 = infer_pth/dir3
pth4 = infer_pth/dir4

model3_files = sorted(list(pth3.glob('*.tif')))
model4_files = sorted(list(pth4.glob('*.tif')))

print('Processing '+str(len(model3_files))+' files...')

Processing 42 files...


In [27]:
@interact
def show_sample(sample=33):
    return visualize_sample(model3_files[sample], model4_files[sample], target_files[sample])

interactive(children=(IntSlider(value=33, description='sample', max=99, min=-33), Output()), _dom_classes=('wi…